In [ ]:
# imports

import os
import sys
import json
import importlib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools
import tools.omstools as omstools
import plotting.plottools as plottools

In [ ]:
# load occupancy from dqmio files

mes = ({
    'PXLayer_1': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1',
    'PXLayer_2': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_2',
    #'PXLayer_3': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_3',
    #'PXLayer_4': 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_4',
})

eras = [
  #'Run2024A-v1', # only commissioning, no lumisections with physics flag set to True
  'Run2024B-v1',
  'Run2024C-v1',
  'Run2024D-v1',
  'Run2024E-v1',
  'Run2024E-v2',
  'Run2024F-v1',
  'Run2024G-v1',
  'Run2024H-v1',
  'Run2024I-v1',
  'Run2024I-v2',
  #'Run2024J-v1'  # pp reference run for heavy ion run; lower pileup and occupancy
  #'Run2025A-v1', # only commissioning (?)
  #'Run2025A-v2', # only commissioning (?)
  'Run2025B-v1',
  'Run2025C-v1',
  'Run2025C-v2',
  'Run2025D-v1',
  'Run2025E-v1',
  'Run2025F-v1',
  'Run2025F-v2',
  'Run2025G-v1'
]

datadir = '/eos/user/l/llambrec/dialstools-output'
dataset = 'ZeroBias'
reco = 'PromptReco'

occupancy_info = {}
for era in eras:
    occupancy_info[era] = {}
    mainera, version = era.split('-')
    for melabel, mename in mes.items():
        f = f'{dataset}-{mainera}-{reco}-{version}-DQMIO-{mename}.parquet'
        f = os.path.join(datadir, f)
        df = iotools.read_parquet(f, columns=['run_number', 'ls_number', 'entries'])
        runs = df['run_number'].values
        lumis = df['ls_number'].values
        entries = df['entries'].values
        occupancy_info[era][melabel] = {'runs': runs, 'lumis': lumis, 'entries': entries}

In [ ]:
# load trigger rates from json file

hltrate_info = {}
for era in eras:
    hltfile = 'omsdata/hltrate_{}.json'.format(era)
    with open(hltfile, 'r') as f:
        hltrate_info[era] = json.load(f)

In [ ]:
# load oms json

oms_info = {}
for era in eras:
    omsfile = 'omsdata/omsdata_{}.json'.format(era)
    with open(omsfile, 'r') as f:
        oms_info[era] = json.load(f)

In [ ]:
# define help functions for plotting

def plot_occupancy_vs_pileup(occupancy_info, oms_info, oms_attr, mes, eras,
                           colors='single', xaxlabel='auto', yaxlabel='Occupancy',
                           ymax=None, normalize = False,
                           physics_flag_filter = False,
                           zero_entries_filter = False,
                           min_entries_filter = None,
                           json_filter = None,
                           dolegend = False):
    # make plot of occupancy vs pileup (or another oms attribute)
    
    # define colors
    if colors=='single':
        # use same color for all eras
        # (typically used if plotting only one era)
        colors = ['b']*len(eras)
    if colors=='perera':
        # use a colormap with different colors for different eras
        # (typically used if plotting multiple eras)
        cmap = mpl.colormaps['viridis']
        colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    
    # initialize figure
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(18,6), squeeze=False)
    axs = axs.flatten()
    
    # loop over monitoring elements and eras
    for meidx, melabel in enumerate(mes.keys()):
        ax = axs[meidx]
        for eraidx, era in enumerate(eras):
            print('Retrieving data for {}, {}'.format(melabel, era))
            
            # get occupancy
            runs = occupancy_info[era][melabel]['runs']
            lumis = occupancy_info[era][melabel]['lumis']
            entries = occupancy_info[era][melabel]['entries']
            print('Found {} lumisections'.format(len(lumis)))
            
            # perform filtering
            if physics_flag_filter:
                physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'physics_flag').astype(bool)
                print('Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
                runs = runs[physics_flag_mask]
                lumis = lumis[physics_flag_mask]
                entries = entries[physics_flag_mask]
            if zero_entries_filter:
                zero_entries_mask = (entries > 0).astype(bool)
                print('Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
                runs = runs[zero_entries_mask]
                lumis = lumis[zero_entries_mask]
                entries = entries[zero_entries_mask]
            if min_entries_filter is not None:
                min_entries_mask = (entries > min_entries_filter).astype(bool)
                print('Min. number of entries filter: {} / {} lumisections passing'.format(np.sum(min_entries_mask), len(min_entries_mask)))
                runs = runs[min_entries_mask]
                lumis = lumis[min_entries_mask]
                entries = entries[min_entries_mask]
            if json_filter is not None:
                json_mask = jsonu.injson(runs, lumis, jsondict=json_filter)
                print('Json filter: {} / {} lumisections passing'.format(np.sum(json_mask), len(json_mask)))
                runs = runs[json_mask]
                lumis = lumis[json_mask]
                entries = entries[json_mask]
            
            # get pileup
            pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], oms_attr)
            
            # make scatter plot
            if normalize:
                entries = np.divide(entries, np.where(pileup>0, pileup, 1))
            ax.scatter(pileup, entries, s=1, alpha=0.25, color=colors[eraidx], label=era)
            if len(eras)==1: ax.text(0.05, 0.95, era + ', ' + melabel, va='top', transform=ax.transAxes, fontsize=12)
        
        # plot aesthetics
        if xaxlabel=='auto': xaxlabel = omsattr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        if ymax is not None: ax.set_ylim((0, ymax))
        if dolegend:
            ncols = len(eras)%4
            leg = ax.legend(loc='upper left', ncols=ncols)
            for handle in leg.legend_handles:
                handle._sizes = [30]
                handle.set_alpha(1)
        fig.subplots_adjust(bottom=-0.2, left=-0.2)
    return fig, axs

In [ ]:
# make plots for pileup

# per era
for era in eras[:1]:
    fig, axs = plot_occupancy_vs_pileup(occupancy_info, oms_info, 'pileup', mes, [era], xaxlabel='Pileup', yaxlabel='Occupancy',
                                      physics_flag_filter=True,
                                      zero_entries_filter=True,
                                      #min_entries_filter=0.02e7
                                     )
    
# eras together
fig, axs = plot_occupancy_vs_pileup(occupancy_info, oms_info, 'pileup', mes, eras, colors='perera', xaxlabel='Pileup', yaxlabel='Occupancy',
                                  ymax=3e7,
                                  dolegend=True,
                                  physics_flag_filter=True,
                                  zero_entries_filter=True,
                                  #min_entries_filter=0.02e7
                                 )
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, '2024/2025 data, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plot for pileup with normalized occupancy

fig, axs = plot_occupancy_vs_pileup(occupancy_info, oms_info, 'pileup', mes, eras,
                                colors='perera', xaxlabel='Pileup', yaxlabel='Occupancy (normalized)',
                                normalize=True,
                                ymax=3e5,
                                dolegend=True,
                                physics_flag_filter=True,
                                zero_entries_filter=True,
                                #min_entries_filter=0.02e7
                                 )
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, '2024/2025 data, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# investigate some cases with zero pileup and high occupancy

era = 'Run2024C-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'pileup')
inds = np.where(((pileup < 5) & (entries > 4e6)))[0]
print(runs[inds])
print(lumis[inds])
print(pileup[inds])
print(entries[inds])
# check these lumisections in OMS:
# both lumi and pileup are indeed zero in OMS for these lumisections, might be OMS error?

In [ ]:
# investigate some cases with high pileup and lower than expected occupancy

era = 'Run2024C-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'pileup')
inds = np.where(((pileup > 45) & (pileup < 60) & (entries > 1e6) & (entries < 2e6)))[0]
print(runs[inds])
print(lumis[inds])
print(entries[inds])
# check run 379456 LS 431 in OMS:
# - lumi and pileup are the same as lumisections before and after, nothing strange.
# - corresponds to drop in trigger rate.

# what happens in other lumisections in same run with trigger rate drop (e.g. 432, 668)?
inds = np.where(((runs == 379456) & (lumis == 668)))[0]
print(runs[inds])
print(lumis[inds])
print(pileup[inds])
print(entries[inds])
# same, just a bit less pronounced (or pileup is lower).

# conclusion: maybe trigger rate is a better normalizer than pileup.

In [ ]:
# make plots for delivered lumi

# per era
for era in eras[:1]:
    fig, axs = plot_occupancy_vs_pileup(
        occupancy_info, oms_info, 'delivered_lumi_per_lumisection', mes, [era], 
        xaxlabel='Delivered luminosity per lumisection (per pb)', 
        yaxlabel='Occupancy',
        physics_flag_filter=True,
        zero_entries_filter=True
    )
    
# eras together
fig, axs = plot_occupancy_vs_pileup(
    occupancy_info, oms_info, 'delivered_lumi_per_lumisection', mes, eras,
    xaxlabel='Delivered luminosity per lumisection (per pb)', 
    yaxlabel='Occupancy',
    colors='perera',
    #ymax=1.3e7,
    dolegend=True,
    physics_flag_filter=True,
    zero_entries_filter=True
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plots for recorded lumi

# per era
for era in eras[:1]:
    fig, axs = plot_occupancy_vs_pileup(
        occupancy_info, oms_info, 'recorded_lumi_per_lumisection', mes, [era], 
        xaxlabel='Recorded luminosity per lumisection (per pb)', 
        yaxlabel='Occupancy',
        physics_flag_filter=True,
        zero_entries_filter=True
    )
    
# eras together
fig, axs = plot_occupancy_vs_pileup(
    occupancy_info, oms_info, 'recorded_lumi_per_lumisection', mes, eras,
    xaxlabel='Recorded luminosity per lumisection (per pb)', 
    yaxlabel='Occupancy',
    colors='perera',
    #ymax=1.3e7,
    dolegend=True,
    physics_flag_filter=True,
    zero_entries_filter=True
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# investigate some cases with high occupancy and low luminosity

era = 'Run2024F-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
lumi = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'delivered_lumi_per_lumisection')
inds = np.where(((lumi < 0.1) & (entries > 2e6) & (runs==383134)))[0]
print(f'Found {len(inds)} instances meeting criteria')
print(f'Run numbers: {runs[inds]}')
print(f'Lumisection numbers: {lumis[inds]}')
print(f'Lumi: {lumi[inds]}')
print(f'Entries: {entries[inds]}')
unique_runs = np.unique(runs[inds])
print(f'Found {len(unique_runs)} unique runs meeting criteria: {unique_runs}')

# run 382209:
# - prescale column 11 (2 bunches)
# - fill 9801 (2 colliding bunches)
# run 382213
# - prescale column 13 (62 bunches) for most of the run
# - fill 9803 (62 colliding bunches)
# run 382216: similar
# run 382229: similar with 362 colliding bunches
# run 382250: similar with 1226 colliding bunches
# run 383134: similar with 362 colliding bunches
# run 383148: similar with 362 colliding bunches

# what is a normal value for colliding bunches and prescale?
# seems to be typically in the order of 2340 bunches.

In [ ]:
# define help functions for plotting

def plot_occupancy_vs_triggerrate(
    occupancy_info, hltrate_info, hltname, mes, eras,
    oms_info=None, physics_flag_filter = False,
    zero_entries_filter = False,
    min_entries_filter = None,
    colors='single', xaxlabel='auto', yaxlabel='Occupancy',
    xmax=None, ymax=None, normalize = False,
    dolegend = False):
    # make plot of occupancy vs trigger rate
    
    # define colors
    if colors=='single':
        # use same color for all eras
        # (typically used if plotting only one era)
        colors = ['b']*len(eras)
    if colors=='perera':
        # use a colormap with different colors for different eras
        # (typically used if plotting multiple eras)
        cmap = mpl.colormaps['viridis']
        colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    
    # initialize figure
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(18,6), squeeze=False)
    axs = axs.flatten()
    
    # loop over monitoring elements and eras
    for meidx, melabel in enumerate(mes.keys()):
        ax = axs[meidx]
        for eraidx, era in enumerate(eras):
            print('Retrieving data for {}, {}'.format(melabel, era))
            
            # get occupancy
            runs = occupancy_info[era][melabel]['runs']
            lumis = occupancy_info[era][melabel]['lumis']
            entries = occupancy_info[era][melabel]['entries']
            print('Found {} lumisections'.format(len(lumis)))
            
            # perform filtering
            if physics_flag_filter:
                if oms_info is None:
                    raise Exception('Must provide OMS info if physics_flag_filter is set to True.')
                physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'physics_flag').astype(bool)
                print('Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
                runs = runs[physics_flag_mask]
                lumis = lumis[physics_flag_mask]
                entries = entries[physics_flag_mask]
            if zero_entries_filter:
                zero_entries_mask = (entries > 0).astype(bool)
                print('Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
                runs = runs[zero_entries_mask]
                lumis = lumis[zero_entries_mask]
                entries = entries[zero_entries_mask]
            if min_entries_filter is not None:
                min_entries_mask = (entries > min_entries_filter).astype(bool)
                print('Min. number of entries filter: {} / {} lumisections passing'.format(np.sum(min_entries_mask), len(min_entries_mask)))
                runs = runs[min_entries_mask]
                lumis = lumis[min_entries_mask]
                entries = entries[min_entries_mask]
            
            # get trigger rate
            values = omstools.find_hlt_rate_for_lumisections(runs, lumis, hltrate_info[era], hltname)
            
            # divide occupancy by trigger rate if requested
            if normalize: entries = np.divide(entries, np.where(values>0, values, 1))
                
            # make the scatter plot
            ax.scatter(values, entries, s=1, alpha=0.25, color=colors[eraidx], label=era)
            if len(eras)==1: ax.text(0.05, 0.95, era + ', ' + melabel, va='top', transform=ax.transAxes, fontsize=12)
        
        # plot aesthetics
        if xaxlabel=='auto': xaxlabel = omsattr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        if xmax is not None: ax.set_xlim((0, xmax))
        if ymax is not None: ax.set_ylim((0, ymax))
        if dolegend:
            ncols = len(eras)%4
            leg = ax.legend(loc='upper left', ncols=ncols)
            for handle in leg.legend_handles:
                handle._sizes = [30]
                handle.set_alpha(1)
        fig.subplots_adjust(bottom=-0.2, left=-0.2)
    return fig, axs

In [ ]:
# make plots for trigger rate

# per era
for era in eras[:1]:
    fig, axs = plot_occupancy_vs_triggerrate(
        occupancy_info, hltrate_info, 'HLT_ZeroBias_v*', mes, [era],
        oms_info=oms_info,
        physics_flag_filter=True,
        zero_entries_filter=True,
        #min_entries_filter=0.02e7,
        xaxlabel='ZeroBias HLT rate', yaxlabel='Occupancy',
    )
    
# eras together
fig, axs = plot_occupancy_vs_triggerrate(
    occupancy_info, hltrate_info, 'HLT_ZeroBias_v*', mes, eras,
    oms_info=oms_info,
    physics_flag_filter=True,
    zero_entries_filter=True,
    #min_entries_filter=0.02e7,
    colors='perera', xaxlabel='ZeroBias HLT rate', yaxlabel='Occupancy',
    xmax=200,
    ymax=3e7,
    dolegend=True,
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, '2024/2025 data, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# investigate some cases with lower than expected occupancy

era = 'Run2024E-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
hltrate = omstools.find_hlt_rate_for_lumisections(runs, lumis, hltrate_info[era], 'HLT_ZeroBias_v*')
inds = np.where(((hltrate>48) & (hltrate < 50) & (entries < 2e6) & (entries > 0)))[0]
print(runs[inds])
print(lumis[inds])
print(hltrate[inds])
print(entries[inds])
# check these lumisections in OMS:
# e.g. run 380963, LS 38-40 -> seems to be correlated with low pileup / luminosity...
# e.g. run 381379, LS 45-49 -> same

# -> probably at least 2 out of trigger rate / pileup / luminosity are needed to explain occupancy

In [ ]:
def plot_occupancy_vs_triggerrate_and_pileup(
    occupancy_info, hltrate_info, hltname, oms_info, oms_attr, mes, eras,
    physics_flag_filter = False,
    zero_entries_filter = False,
    min_entries_filter = None,
    colors='single', xaxlabel='auto', yaxlabel='auto', zaxlabel='Occupancy',
    zmax=None,
    dolegend = False):
    # make 3D plot of occupancy vs trigger rate and pileup
    
    # define colors
    if colors=='single':
        # use same color for all eras
        # (typically used if plotting only one era)
        colors = ['b']*len(eras)
    if colors=='perera':
        # use a colormap with different colors for different eras
        # (typically used if plotting multiple eras)
        cmap = mpl.colormaps['viridis']
        colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    
    # loop over monitoring elements and eras
    for meidx, melabel in enumerate(mes.keys()):
        
        # initialize figure
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        
        for eraidx, era in enumerate(eras):
            print('Retrieving data for {}, {}'.format(melabel, era))
            
            # get occupancy
            runs = occupancy_info[era][melabel]['runs']
            lumis = occupancy_info[era][melabel]['lumis']
            entries = occupancy_info[era][melabel]['entries']
            print('Found {} lumisections'.format(len(lumis)))
            
            # perform filtering
            if physics_flag_filter:
                if oms_info is None:
                    raise Exception('Must provide OMS info if physics_flag_filter is set to True.')
                physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'physics_flag').astype(bool)
                print('Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
                runs = runs[physics_flag_mask]
                lumis = lumis[physics_flag_mask]
                entries = entries[physics_flag_mask]
            if zero_entries_filter:
                zero_entries_mask = (entries > 0).astype(bool)
                print('Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
                runs = runs[zero_entries_mask]
                lumis = lumis[zero_entries_mask]
                entries = entries[zero_entries_mask]
            if min_entries_filter is not None:
                min_entries_mask = (entries > min_entries_filter).astype(bool)
                print('Min. number of entries filter: {} / {} lumisections passing'.format(np.sum(min_entries_mask), len(min_entries_mask)))
                runs = runs[min_entries_mask]
                lumis = lumis[min_entries_mask]
                entries = entries[min_entries_mask]
            
            # get trigger rate
            hltrate = omstools.find_hlt_rate_for_lumisections(runs, lumis, hltrate_info[era], hltname)
            
            # get pileup 
            pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], oms_attr)
                
            # make the scatter plot
            ax.scatter(hltrate, pileup, entries, s=1, alpha=0.25, color=colors[eraidx], label=era)
        
        # plot aesthetics
        if xaxlabel=='auto': xaxlabel = hltname
        if yaxlabel=='auto': yaxlabel = oms_attr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        ax.set_zlabel(zaxlabel, fontsize=15)
        if zmax is not None: ax.set_zlim((0, zmax))
        #if dolegend:
        #    ncols = len(eras)%4
        #    leg = ax.legend(loc='upper left', ncols=ncols)
        #    for handle in leg.legend_handles:
        #        handle._sizes = [30]
        #        handle.set_alpha(1)
    return fig, axs

In [ ]:
# per era

for era in eras[:1]:
    fig, axs = plot_occupancy_vs_triggerrate_and_pileup(
        occupancy_info, hltrate_info, 'HLT_ZeroBias_v*', oms_info, 'pileup', mes, [era],
        physics_flag_filter=True,
        zero_entries_filter=True,
        #min_entries_filter=0.02e7,
        xaxlabel='ZeroBias HLT rate', yaxlabel='Pileup', zaxlabel='Occupancy',
    )
    
# eras together
fig, axs = plot_occupancy_vs_triggerrate_and_pileup(
    occupancy_info, hltrate_info, 'HLT_ZeroBias_v*', oms_info, 'pileup', mes, eras,
    physics_flag_filter=True,
    zero_entries_filter=True,
    #min_entries_filter=0.02e7,
    colors='perera', xaxlabel='ZeroBias HLT rate', yaxlabel='Pileup', zaxlabel='Occupancy',
    dolegend=True,
)

In [ ]:
# find a suitable normalization function

from scipy.optimize import curve_fit

def linear_2d(points, a, b, c):
    x, y = points
    prefactor = 1e7
    # note: prefactor is hard-coded to help the fit stability
    #       by making the coefficients more in the order of unity.
    z = prefactor*(a*x + b*y + c)
    return z

def product_2d(points, a):
    x, y = points
    prefactor = 1e7
    z = prefactor * a * np.multiply(x, y)
    return z

def product_2d_softtail(points, a):
    x, y = points
    prefactor = 1e7
    # calculate the product of x and y and do standard scaling
    product = np.multiply(x, y)
    maxproduct = np.amax(product)
    product /= maxproduct
    # settings
    scale = 0.01
    offset = 0.01
    z = prefactor * a * (product + (offset*scale)/(scale+product))
    return z

def product_3d(points, a):
    x, y, z = points
    prefactor = 1e7
    n = prefactor * a * np.multiply(np.multiply(x, y), z)
    return n

def quadratic_3d(points, a, b, c, d, e, f, g, h, i):
    x, y, z = points
    prefactor = 1e7
    n = prefactor * (a*x + b*y + c*z + d*np.square(x) + e*np.square(y) + f*np.square(z)
                     + g*np.multiply(x, y) + h*np.multiply(y, z) + i*np.multiply(x, z))
    return n

# loop over mes and eras
norm_info = {}
for era in eras:
    norm_info[era] = {}
    for melabel in mes.keys():
        
        # get the data
        runs = occupancy_info[era][melabel]['runs']
        lumis = occupancy_info[era][melabel]['lumis']
        entries = occupancy_info[era][melabel]['entries']
        print('Found {} lumisections'.format(len(lumis)))
        
        # perform some filtering by physics flag
        physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'physics_flag').astype(bool)
        print('Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
        runs = runs[physics_flag_mask]
        lumis = lumis[physics_flag_mask]
        entries = entries[physics_flag_mask]
        
        # remove zero entries
        zero_entries_mask = (entries > 0).astype(bool)
        print('Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
        runs = runs[zero_entries_mask]
        lumis = lumis[zero_entries_mask]
        entries = entries[zero_entries_mask]
        
        # set minimum number of entries
        #min_entries_mask = (entries > 0.02e7).astype(bool)
        #print('Min. number of entries filter: {} / {} lumisections passing'.format(np.sum(min_entries_mask), len(min_entries_mask)))
        #runs = runs[min_entries_mask]
        #lumis = lumis[min_entries_mask]
        #entries = entries[min_entries_mask]
        
        # get trigger rate
        hltrate = omstools.find_hlt_rate_for_lumisections(runs, lumis, hltrate_info[era], 'HLT_ZeroBias_v*', verbose=False)
            
        # get pileup 
        pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'pileup')
        
        # get lumi
        lumi = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'recorded_lumi_per_lumisection')
        
        # do the fit
        fitfunc = product_2d_softtail
        fitresult = curve_fit(fitfunc, (hltrate, pileup), entries)
        #fitfunc = product_3d
        #fitresult = curve_fit(fitfunc, (hltrate, pileup, lumi), entries)
        bestcoeffs = fitresult[0]
        print(f'Best coefficients: {bestcoeffs}')
        
        # make prediction
        norm = fitfunc((hltrate, pileup), *bestcoeffs)
        #norm = fitfunc((hltrate, pileup, lumi), *bestcoeffs)
        norm_info[era][melabel] = {
            'run_number': runs,
            'lumisection_number': lumis,
            'norm': norm
        }

In [ ]:
def plot_occupancy_vs_norm(
    occupancy_info, norm_info, norm_attr, mes, eras,
    oms_info=None, physics_flag_filter = False,
    zero_entries_filter = False,
    min_entries_filter = None,
    colors='single', xaxlabel='auto', yaxlabel='Occupancy',
    ymax=None, normalize = False,
    dolegend = False):
    # make plot of occupancy vs custom normalization
    
    # define colors
    if colors=='single':
        # use same color for all eras
        # (typically used if plotting only one era)
        colors = ['b']*len(eras)
    if colors=='perera':
        # use a colormap with different colors for different eras
        # (typically used if plotting multiple eras)
        cmap = mpl.colormaps['viridis']
        colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    
    # initialize figure
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(18,6), squeeze=False)
    axs = axs.flatten()
    
    # loop over monitoring elements and eras
    for meidx, melabel in enumerate(mes.keys()):
        ax = axs[meidx]
        for eraidx, era in enumerate(eras):
            print('Retrieving data for {}, {}'.format(melabel, era))
            
            # get occupancy
            runs = occupancy_info[era][melabel]['runs']
            lumis = occupancy_info[era][melabel]['lumis']
            entries = occupancy_info[era][melabel]['entries']
            print('Found {} lumisections'.format(len(lumis)))
            
            # perform filtering
            if physics_flag_filter:
                if oms_info is None:
                    raise Exception('Must provide OMS info if physics_flag_filter is set to True.')
                physics_flag_mask = omstools.find_oms_attr_for_lumisections(runs, lumis, oms_info[era], 'physics_flag').astype(bool)
                print('Physics flag filter: {} / {} lumisections passing'.format(np.sum(physics_flag_mask), len(physics_flag_mask)))
                runs = runs[physics_flag_mask]
                lumis = lumis[physics_flag_mask]
                entries = entries[physics_flag_mask]
            if zero_entries_filter:
                zero_entries_mask = (entries > 0).astype(bool)
                print('Zero number of entries filter: {} / {} lumisections passing'.format(np.sum(zero_entries_mask), len(zero_entries_mask)))
                runs = runs[zero_entries_mask]
                lumis = lumis[zero_entries_mask]
                entries = entries[zero_entries_mask]
            if min_entries_filter is not None:
                min_entries_mask = (entries > min_entries_filter).astype(bool)
                print('Min. number of entries filter: {} / {} lumisections passing'.format(np.sum(min_entries_mask), len(min_entries_mask)))
                runs = runs[min_entries_mask]
                lumis = lumis[min_entries_mask]
                entries = entries[min_entries_mask]
            
            # get norm
            values = omstools.find_oms_attr_for_lumisections(runs, lumis, norm_info[era][melabel], norm_attr)
            
            # divide occupancy by trigger rate if requested
            if normalize: entries = np.divide(entries, np.where(values>0, values, 1))
                
            # make the scatter plot
            ax.scatter(values, entries, s=1, alpha=0.25, color=colors[eraidx], label=era)
            if len(eras)==1: ax.text(0.05, 0.95, era + ', ' + melabel, va='top', transform=ax.transAxes, fontsize=12)
        
        # plot aesthetics
        if xaxlabel=='auto': xaxlabel = omsattr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        if ymax is not None: ax.set_ylim((0, ymax))
        if dolegend:
            ncols = len(eras)%4
            leg = ax.legend(loc='upper left', ncols=ncols)
            for handle in leg.legend_handles:
                handle._sizes = [30]
                handle.set_alpha(1)
        fig.subplots_adjust(bottom=-0.2, left=-0.2)
    return fig, axs

In [ ]:
# make plots for custom norm

# per era
for era in eras[:1]:
    fig, axs = plot_occupancy_vs_norm(
        occupancy_info, norm_info, 'norm', mes, [era],
        oms_info=oms_info,
        physics_flag_filter=True,
        zero_entries_filter=True,
        #min_entries_filter=0.02e7,
        xaxlabel='Norm', yaxlabel='Occupancy',
    )
    
# eras together
fig, axs = plot_occupancy_vs_norm(
    occupancy_info, norm_info, 'norm', mes, eras,
    oms_info=oms_info,
    physics_flag_filter=True,
    zero_entries_filter=True,
    #min_entries_filter=0.02e7,
    colors='perera', xaxlabel='Norm', yaxlabel='Occupancy',
    #ymax=1e7,
    dolegend=True,
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plots for custom norm

# normalized
fig, axs = plot_occupancy_vs_norm(
    occupancy_info, norm_info, 'norm', mes, eras,
    oms_info=oms_info,
    physics_flag_filter=True,
    zero_entries_filter=True,
    #min_entries_filter=0.02e7,
    normalize=True,
    colors='perera', xaxlabel='Norm', yaxlabel='Occupancy',
    ymax = 3, dolegend=True,
)
for idx,ax in enumerate(axs): ax.text(0.95, 0.95, 'Run 2024, ' + list(mes.keys())[idx], ha='right', va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# investigate some cases with lower occupancy than expected

era = 'Run2024G-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
norm = omstools.find_oms_attr_for_lumisections(runs, lumis, norm_info[era][melabel], 'norm')
normalized_entries = np.divide(entries, np.where(norm>0, norm, 1))
inds = np.where(((norm > 3e6) & (normalized_entries < 0.75)))[0]
print(f'Found {len(inds)} instances meeting criteria')
print(f'Run numbers: {runs[inds]}')
print(f'Lumisection numbers: {lumis[inds]}')
print(f'Norm: {norm[inds]}')
print(f'Entries: {entries[inds]}')
unique_runs = np.unique(runs[inds])
print(f'Found {len(unique_runs)} unique runs meeting criteria: {unique_runs}')

# run 383944: 9 colliding bunches
# run 383945: same

In [ ]:
# investigate some cases with higher occupancy than expected

era = 'Run2024F-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
norm = omstools.find_oms_attr_for_lumisections(runs, lumis, norm_info[era][melabel], 'norm')
normalized_entries = np.divide(entries, np.where(norm>0, norm, 1))
inds = np.where(((norm > 1e6) & (norm < 1.5e6) & (normalized_entries > 1.5)))[0]
print(f'Found {len(inds)} instances meeting criteria')
print(f'Run numbers: {runs[inds]}')
print(f'Lumisection numbers: {lumis[inds]}')
print(f'Norm: {norm[inds]}')
print(f'Entries: {entries[inds]}')
unique_runs = np.unique(runs[inds])
print(f'Found {len(unique_runs)} unique runs meeting criteria: {unique_runs}')

# run 382213: see earlier: 62 bunches
# run 382834: only one LS, maybe to investigate later
# run 382937: only one LS, maybe to investigate later
# run 383134: see earlier: 362 bunches
# run 383662: only one LS, maybe to investigate later

In [ ]:
# investigate some more cases with even higher occupancy

era = 'Run2024F-v1'
melabel = 'PXLayer_1'
runs = occupancy_info[era][melabel]['runs']
lumis = occupancy_info[era][melabel]['lumis']
entries = occupancy_info[era][melabel]['entries']
norm = omstools.find_oms_attr_for_lumisections(runs, lumis, norm_info[era][melabel], 'norm')
normalized_entries = np.divide(entries, np.where(norm>0, norm, 1))
inds = np.where(((norm < 0.5e6) & (norm > 0.1e6) & (normalized_entries > 2)))[0]
print(f'Found {len(inds)} instances meeting criteria')
print(f'Run numbers: {runs[inds]}')
print(f'Lumisection numbers: {lumis[inds]}')
print(f'Norm: {norm[inds]}')
print(f'Entries: {entries[inds]}')
unique_runs = np.unique(runs[inds])
print(f'Found {len(unique_runs)} unique runs meeting criteria: {unique_runs}')

# e.g. run 382255, LS 42: unclear. there seems to be an instability in pileup and lumi, but not clear why occupancy is higher than expected.